In [4]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [5]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/studios"
requestURL = requests.get(URL)

In [6]:
content = requestURL.content

In [7]:
soup = BeautifulSoup(content, "html.parser")

In [8]:
rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]



In [69]:
dict = {'Monthly': rent, 'Number of baths': bath}
df= pd.DataFrame(dict)

In [70]:
df['Monthly'] = df['Monthly'].str.replace('\D+', '')

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_16796/424978522.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = df['Monthly'].str.replace('\D+', '')


In [71]:
df

,Monthly,Number of baths
0,1900,Studio • 1 Bath
1,2369,Studio • 1 Bath
2,2268,Studio • 1 Bath
3,2570,Studio • 1 Bath
4,2650,Studio • 1 Bath
5,2450,Studio • 1 Bath
6,2495,Studio • 1 Bath
7,3500,Studio • 1 Bath
8,2700,Studio • 1 Bath
9,2375,Studio • 1 Bath


In [12]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/studios/k0c37l1700292?rb=true"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

soup2 = BeautifulSoup(content2, "html.parser")

In [112]:
main_container = soup2.find("div",id="MainContainer")
main_containerContent = main_container.find("div", id="mainPageContent")
realEstate = main_containerContent.find("div", class_="layout-3 new-real-estate-srp")
realEstate2 = realEstate.find("div", class_="col-2 new-real-estate-srp")
realEstate3 = realEstate2.findAll("div", class_="container-results large-images")[1]
realEstate4 = [x.find('div', class_="info") for x in realEstate3.findAll("div", attrs={"data-listing-id":True})]

realEstatePrices = [x.find('div', class_=["price"]) for x in realEstate4]
realEstatePrices = [x.text.strip() for x in realEstatePrices]

In [113]:
realEstatePrices

['$2,369.00',
 '$2,650.00',
 '$1,640.00',
 '$2,369.00',
 '$2,195.00',
 '$2,375.00',
 '$3,485.00',
 '$2,495.00',
 '$2,325.00',
 '$2,900.00',
 '$2,268.00',
 '$2,570.00',
 '$2,510.00',
 '$2,650.00',
 '$2,450.00',
 '$2,450.00',
 '$2,480.00',
 '$2,980.00',
 '$2,400.00',
 '$2,200.00',
 '$2,600.00']